In [1]:
import pandas as pd

In [2]:
annot = pd.read_table('../data/annotation.txt')

In [3]:
annot.columns =[ele.lower().replace(' ','_') for ele in annot.columns]

In [4]:
annot = annot[['ccle_name','gender','site_primary','histology','hist_subtype1']].drop_duplicates()

In [5]:
annot = annot.reset_index()

In [6]:
annot = annot.rename(columns = {'index':'id','ccle_name':'celllinename'})

In [7]:
expr = pd.read_table('../data/CCLE_Expression_Entrez_2012-09-29.gct', skiprows = 2)

/Library/Python/2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
expr = expr.dropna()

In [9]:
expr = expr.ix[~expr.Description.duplicated(),:]

In [10]:
del expr['Name']

In [11]:
expr = pd.melt(expr, id_vars = ['Description'], var_name = "celllinename", value_name = "expression").rename(columns = {'Description':'symbol'})

In [12]:
expr = expr.merge(annot[['celllinename','id']], on = 'celllinename')

In [13]:
expr = expr.rename(columns = {'id':'annotation_id'})

In [14]:
import sqlite3 as db

In [15]:
con = db.connect('../ccle.db')

In [17]:
expr.to_sql('expression', con = con, if_exists='replace', index_label = 'id')

In [16]:
annot.to_sql('annotation', con = con, if_exists='replace', index = False)

In [18]:
con.execute('ALTER TABLE annotation ADD PRIMARY KEY (id)')

OperationalError: near "PRIMARY": syntax error

In [20]:
con.execute('CREATE INDEX symbol_idx ON expression (symbol)')